In [4]:
import pandas as pd
import streamlit as st
from helpers.iaps_helper import *
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

path = './data/iaps/iaps.csv'
df = pd.read_csv(path)
## Transform dataframe with assign and pipe
def tweak_df_iaps(df):
     currencies=fetch_exchange_rates()
     return(df
            .assign(
               revenue_usd = lambda df_: (df_.local_revenue_gross 
                              / df_.local_currency_code.map(currencies)
                              ).fillna(0), 
               event_time = lambda df_: pd.to_datetime(df_.event_time),
               date = lambda df_:pd.to_datetime(df_.date, format='%Y-%m-%d'),
               install_date = lambda df_: pd.to_datetime(df_.install_date, format='%Y-%m-%d'),
               install_datetime = lambda df_: pd.to_datetime(df_.install_datetime)
          ).loc[:,['event_time', 'client_country', 'date', 
               'install_date','progress_id','retention_day',
               'transaction_id','price','local_currency_code',
               'local_revenue_gross','ab_test_variants','revenue_usd']])
df.pipe(tweak_df_iaps).sample(10)

<Response [200]>


,event_time,client_country,date,install_date,progress_id,retention_day,transaction_id,price,local_currency_code,local_revenue_gross,ab_test_variants,revenue_usd
560,2024-01-24 17:04:35.668000+00:00,US,2024-01-24,2024-01-16,c35eb330-fa98-42d7-8dcf-ccc514f0be9a,8,GPA.3379-2730-4958-46261,0.99,USD,0.99,No AB Test,0.990000
865,2024-02-18 22:58:10.262000+00:00,US,2024-02-18,2024-02-04,eae86baa-6757-48bf-938b-e811017c4a32,14,GPA.3373-5613-4289-36897,1.99,USD,1.99,Variant A,1.990000
464,2024-01-16 03:12:59.704000+00:00,US,2024-01-16,2024-01-09,4e7b57c4-8b10-48c7-885c-dae754f1d03d,7,GPA.3376-9185-1761-89916,1.99,USD,1.99,No AB Test,1.990000
361,2024-01-09 02:47:00.784000+00:00,US,2024-01-09,2024-01-06,ad97cb4e-b047-4f66-ac1b-44eb2e0b764e,3,GPA.3365-2778-5045-51231,3.49,USD,3.49,No AB Test,3.490000
113,2023-11-08 09:35:56.881000+00:00,NL,2023-11-08,2023-11-04,11ffe36a-5aa1-423b-8004-d0e969ae5d58,4,GPA.3351-2314-4607-28784,2.29,EUR,2.29,No AB Test,2.467518
82,2023-11-02 03:58:46.304000+00:00,US,2023-11-02,2023-11-02,d86963d9-99e5-4589-8a8e-240ffa762f6b,0,GPA.3393-7598-7282-86854,0.99,USD,0.99,No AB Test,0.990000
809,2024-02-13 03:07:45.561000+00:00,CA,2024-02-13,2023-11-13,37face90-e950-4552-808d-7f4f0d4bee97,92,GPA.3389-2935-0738-39730,2.89,CAD,2.89,Control,2.140802
776,2024-02-09 06:46:50.568000+00:00,US,2024-02-09,2024-02-09,615463d2-f451-45a4-91bd-ebb7b8d04ccb,0,GPA.3315-6810-6790-14166,0.99,USD,0.99,No AB Test,0.990000
210,2023-11-22 21:33:41.376000+00:00,GB,2023-11-22,2023-11-22,2b092aa2-db65-498c-b7bf-da47d762f27e,0,GPA.3327-9741-4426-92221,1.79,GBP,1.79,No AB Test,2.254090
40,2023-10-26 07:32:30.551000+00:00,US,2023-10-26,2023-10-26,b1dc2787-99c8-4050-a857-6566076afeab,0,GPA.3392-9982-4107-85459,1.99,USD,1.99,No AB Test,1.990000


In [30]:
import requests

try:
    response = requests.get('https://www.floatrates.com/daily/usd.json')
    response.raise_for_status()
    print(response)
except requests.exceptions.HTTPError as err:
    raise SystemExit(err)

response_json = response.json()
currencies = {response_json[i]['code']:response_json[i]['rate'] for i in response_json}
currencies['USD'] =1
currencies


<Response [200]>


{'EUR': 0.92805788904752,
 'GBP': 0.79411221071112,
 'JPY': 150.27487348263,
 'AUD': 1.5305337872185,
 'CHF': 0.881295702289,
 'CAD': 1.3499614057536,
 'DKK': 6.9115546259392,
 'CNY': 7.1973694994215,
 'VND': 24536.705121585,
 'IQD': 1335.1712352654,
 'CRC': 515.09973684209,
 'ILS': 3.6278054482261,
 'PEN': 3.8246179809187,
 'LYD': 4.9209125964011,
 'PYG': 7297.8723404251,
 'MXN': 17.041045526431,
 'BYN': 3.2726251754797,
 'TND': 3.1542579453283,
 'BGN': 1.8129342161874,
 'PHP': 56.110325471777,
 'KZT': 450.94937454357,
 'RSD': 108.56325871438,
 'PLN': 4.0099821586168,
 'ZAR': 18.952089705466,
 'AZN': 1.69899750152,
 'JOD': 0.70927818901031,
 'TRY': 30.868695213325,
 'TWD': 31.486988815344,
 'IRR': 42807.290210768,
 'COP': 3919.9999999999,
 'IDR': 15657.497633384,
 'NGN': 1605.82,
 'LBP': 52919.708466453,
 'PAB': 1,
 'SEK': 10.422119811305,
 'KRW': 1337.1724681628,
 'BDT': 110.92312530295,
 'AED': 3.6787802526263,
 'BOB': 6.8599999999998,
 'DOP': 58.90661175618,
 'NZD': 1.6284294613818

In [9]:
# Ads


columns= ads2.columns

pd.concat([ads1,ads2]).reset_index(drop=True)

,event_time,created_cik,date,created_date,first_date,install_date,client_country,progress_id,app_version,user_id,...,revenue,revenue_precision,request_reason,show_reason,ad_placement,split,ab_test_variants,retention_day,last_interaction_date,days_inactive
0,2024-01-22 11:56:57.503 UTC,2023-12-03 11:55:09 UTC,2024-01-22,2023-12-03,2024-01-22,2023-12-03,US,37342849-43be-4f5e-ac47-559fa82e0e8e,1.7.0,00006eee-052b-4c63-888d-246a47cc61dc,...,0.028236,exact,interstitial,interstitial,interstitial,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,50,2024-01-22,29
1,2024-01-26 21:16:39.712 UTC,2023-12-03 11:55:09 UTC,2024-01-26,2023-12-03,2024-01-22,2023-12-03,US,37342849-43be-4f5e-ac47-559fa82e0e8e,1.7.0,00006eee-052b-4c63-888d-246a47cc61dc,...,0.017053,estimated,interstitial,interstitial,interstitial,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,54,2024-01-26,25
2,2024-01-26 21:19:05.234 UTC,2023-12-03 11:55:09 UTC,2024-01-26,2023-12-03,2024-01-22,2023-12-03,US,37342849-43be-4f5e-ac47-559fa82e0e8e,1.7.0,00006eee-052b-4c63-888d-246a47cc61dc,...,0.016726,estimated,init_app,rewarded,out_of_moves_ads,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,54,2024-01-26,25
3,2024-01-26 21:19:53.419 UTC,2023-12-03 11:55:09 UTC,2024-01-26,2023-12-03,2024-01-22,2023-12-03,US,37342849-43be-4f5e-ac47-559fa82e0e8e,1.7.0,00006eee-052b-4c63-888d-246a47cc61dc,...,0.016726,estimated,dismissed,rewarded,out_of_moves_ads,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,54,2024-01-26,25
4,2024-01-26 21:20:46.157 UTC,2023-12-03 11:55:09 UTC,2024-01-26,2023-12-03,2024-01-22,2023-12-03,US,37342849-43be-4f5e-ac47-559fa82e0e8e,1.7.0,00006eee-052b-4c63-888d-246a47cc61dc,...,0.016726,estimated,dismissed,rewarded,out_of_moves_ads,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,54,2024-01-26,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501936,2024-02-14 02:58:47.931 UTC,2024-02-08 07:21:08 UTC,2024-02-14,2024-02-08,2024-02-08,2024-02-08,US,8b310e0b-d49e-4753-ae10-6904f650f00c,1.8.6,68fb2882-4756-4045-b324-814ff254295b,...,0.023316,estimated,dismissed,rewarded,double_reward,"{""bundle_contents"":1700,""bundle_purchase"":1800...",Control,6,2024-02-14,6
1501937,2024-02-14 03:01:44.751 UTC,2024-02-08 07:21:08 UTC,2024-02-14,2024-02-08,2024-02-08,2024-02-08,US,8b310e0b-d49e-4753-ae10-6904f650f00c,1.8.6,68fb2882-4756-4045-b324-814ff254295b,...,0.023316,estimated,dismissed,rewarded,out_of_moves_ads,"{""bundle_contents"":1700,""bundle_purchase"":1800...",Control,6,2024-02-14,6
1501938,2024-02-14 03:02:30.44 UTC,2024-02-08 07:21:08 UTC,2024-02-14,2024-02-08,2024-02-08,2024-02-08,US,8b310e0b-d49e-4753-ae10-6904f650f00c,1.8.6,68fb2882-4756-4045-b324-814ff254295b,...,0.023316,estimated,dismissed,rewarded,double_reward,"{""bundle_contents"":1700,""bundle_purchase"":1800...",Control,6,2024-02-14,6
1501939,2024-02-14 03:02:43.88 UTC,2024-02-08 07:21:08 UTC,2024-02-14,2024-02-08,2024-02-08,2024-02-08,US,8b310e0b-d49e-4753-ae10-6904f650f00c,1.8.6,68fb2882-4756-4045-b324-814ff254295b,...,0.023896,estimated,interstitial,interstitial,interstitial,"{""bundle_contents"":1700,""bundle_purchase"":1800...",Control,6,2024-02-14,6


In [29]:
# Ads

import pandas as pd 
import numpy as np
import streamlit as st
from helpers.iaps_helper import *
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

ads1=pd.read_csv('./data/ads/ads.csv')
ads2 =pd.read_csv('./data/ads/ads2.csv')

ads = pd.concat([ads1,ads2])

ads = ads.reset_index(drop=True).assign(event_time = 
                                        lambda df_:pd.to_datetime(df_.event_time.str.replace('UTC', '').str.strip(), 
                                                                format='ISO8601'),
                                        created_cik = 
                                        lambda df_:pd.to_datetime(df_.created_cik.str.replace('UTC', '').str.strip(), 
                                                                format='ISO8601'),
                                        date = lambda df_:pd.to_datetime(df_.date),
                                        created_date = lambda df_:pd.to_datetime(df_.created_date),
                                        install_date = lambda df_:pd.to_datetime(df_.install_date),
                                        first_date = lambda df_:pd.to_datetime(df_.first_date),
                                        last_interaction_date = lambda df_:pd.to_datetime(df_.last_interaction_date)
                                        
                                        ).assign(
                                            created_date = lambda df_: df_.created_date.where(df_.created_date != df_.created_date.min(), df_.first_date),
                                            install_date = lambda df_: df_.install_date.where(df_.install_date != df_.install_date.min(), df_.first_date),
                                            retention_day = lambda df_: ((df_.date - df_.created_date) / np.timedelta64(1,'D')).astype('int')
                                        )
    

daily_impressions = ads.groupby('date')['event'].count()

ads[ads.client_country.isnull()]



,event_time,created_cik,date,created_date,first_date,install_date,client_country,progress_id,app_version,user_id,...,revenue,revenue_precision,request_reason,show_reason,ad_placement,split,ab_test_variants,retention_day,last_interaction_date,days_inactive
48957,2024-01-25 16:58:33.697,2024-01-23 18:41:34,2024-01-25,2024-01-23,2024-01-25,2024-01-23,NaN,385a56bc-1bb6-46c1-8d60-4ba08a550d33,1.7.0,2fd43739-1608-4ec9-8891-55897b5d17f6,...,0.001778,exact,interstitial,interstitial,interstitial,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,2,2024-01-25,26
48958,2024-01-28 09:58:31.918,2024-01-23 18:41:34,2024-01-28,2024-01-23,2024-01-25,2024-01-23,NaN,385a56bc-1bb6-46c1-8d60-4ba08a550d33,1.7.0,2fd43739-1608-4ec9-8891-55897b5d17f6,...,0.002246,exact,interstitial,interstitial,interstitial,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,5,2024-01-28,23
48959,2024-01-28 10:04:44.028,2024-01-23 18:41:34,2024-01-28,2024-01-23,2024-01-25,2024-01-23,NaN,385a56bc-1bb6-46c1-8d60-4ba08a550d33,1.7.0,2fd43739-1608-4ec9-8891-55897b5d17f6,...,0.001354,exact,interstitial,interstitial,interstitial,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,5,2024-01-28,23
54225,2024-01-20 08:52:12.963,2024-01-20 08:49:01,2024-01-20,2024-01-20,2024-01-20,2024-01-20,NaN,2470709e-39fa-4e5a-b67f-e3783bc4d852,1.7.0,317413f9-4777-45f7-90ce-0af69a9aefa5,...,0.001018,exact,interstitial,interstitial,interstitial,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,0,2024-01-20,31
54226,2024-01-20 08:58:42.225,2024-01-20 08:49:01,2024-01-20,2024-01-20,2024-01-20,2024-01-20,NaN,2470709e-39fa-4e5a-b67f-e3783bc4d852,1.7.0,317413f9-4777-45f7-90ce-0af69a9aefa5,...,0.001231,exact,interstitial,interstitial,interstitial,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,0,2024-01-20,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455399,2024-02-06 12:28:02.414,2024-01-20 08:49:01,2024-02-06,2024-01-20,2024-02-01,2024-01-20,NaN,2470709e-39fa-4e5a-b67f-e3783bc4d852,1.7.0,317413f9-4777-45f7-90ce-0af69a9aefa5,...,0.000414,exact,dismissed,rewarded,out_of_moves_ads,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,17,2024-02-06,15
1455400,2024-02-06 12:34:20.203,2024-01-20 08:49:01,2024-02-06,2024-01-20,2024-02-01,2024-01-20,NaN,2470709e-39fa-4e5a-b67f-e3783bc4d852,1.7.0,317413f9-4777-45f7-90ce-0af69a9aefa5,...,0.000352,exact,init_app,rewarded,out_of_moves_ads,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,17,2024-02-06,15
1455401,2024-02-06 12:34:51.598,2024-01-20 08:49:01,2024-02-06,2024-01-20,2024-02-01,2024-01-20,NaN,2470709e-39fa-4e5a-b67f-e3783bc4d852,1.7.0,317413f9-4777-45f7-90ce-0af69a9aefa5,...,0.000457,exact,dismissed,rewarded,out_of_moves_ads,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,17,2024-02-06,15
1455402,2024-02-06 12:37:33.479,2024-01-20 08:49:01,2024-02-06,2024-01-20,2024-02-01,2024-01-20,NaN,2470709e-39fa-4e5a-b67f-e3783bc4d852,1.7.0,317413f9-4777-45f7-90ce-0af69a9aefa5,...,0.000995,exact,interstitial,interstitial,interstitial,"{""bundle_contents"":1700,""bundle_purchase"":1701...",No AB Test,17,2024-02-06,15


In [22]:
print(ads.progress_id.nunique())
print(ads.user_id.nunique())
print('user_id with nulls', ads[ads.progress_id.isnull()].user_id.nunique)

ads[ads.progress_id.isnull()].groupby('user_id').agg({'progress_id':'nunique'})

28113
28508
user_id with nulls 27


,progress_id
user_id,
0769e17e-c434-424e-9bfa-aa30afa0e38f,0
18c1057d-9681-423f-8655-b6526e03a8a1,0
1c7a50e1-26be-49aa-8f29-08054ebea82c,0
31a54a4f-9ab5-4f44-8e4c-0f1f235ede4e,0
338cddca-9601-489b-9dcc-5000a70ee3ad,0
34675991-1b56-4b10-bf3e-061c5760579e,0
41d85674-488b-42dc-bfb2-18154f6abe91,0
4f3bebd4-1384-4ea5-9cd0-d7124ad5d113,0
4f96fecf-30c5-48d3-ae67-4468f6227ecd,0


In [33]:
def create_daily_plot(df: pd.DataFrame, groups: list[str], metric: str, function: list[str]) -> px.line:
    """
    Create a daily line plot from a pandas DataFrame.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        groups (list[str]): List of column names to group by.
        metric (str): The column name of the metric to plot.
        function (list[str]): List of aggregation functions to apply.
        
    Returns:
        px.line: A Plotly line plot.
    """
    try:
        grouped = df.groupby(groups).agg({metric: function}).reset_index()
        grouped.columns = groups + [metric]
        
        if len(groups) == 2:
            line = px.line(grouped, x=groups[0], y=metric, color=groups[1])
        elif len(groups) == 1:
            line = px.line(grouped, x=groups[0], y=metric)
            
        line.update_traces(mode='markers+lines')
        line.update_layout(legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
        ))
        
        return line
    
    except Exception as e:
        print(f"An error occurred: {e}")
    

In [36]:
type(ads)

pandas.core.frame.DataFrame

In [59]:
# Coins 
import pandas as pd
from helpers.coins import *
coins = './data/coins/coins.csv'
df= pd.read_csv(coins).

def tweak_df(df):
    return (
        df.assign(
        created_date = lambda df_: pd.to_datetime(df_.created_date,format='%Y-%m-%d'),
        date = lambda df_:pd.to_datetime(df_.date,format='%Y-%m-%d'),
        event_time = lambda df_:pd.to_datetime(df_.event_time.str.replace(' UTC',''), format="ISO8601"))
        )

# Daily coins flow
daily_coins = df.assign(
    created_date = lambda df_: pd.to_datetime(df_.created_date,format='%Y-%m-%d'),
    date = lambda df_:pd.to_datetime(df_.date,format='%Y-%m-%d'),
    event_time = lambda df_:pd.to_datetime(df_.event_time.str.replace(' UTC',''), format="ISO8601")
).groupby(['date','event'])['coins'].sum().reset_index()

fig = px.bar(daily_coins, x='date', y='coins',color='event')




ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'event=reward:collected<br>date=%{x}<br>coins=%{y}<extra></extra>',
              'legendgroup': 'reward:collected',
              'marker': {'color': '#000001', 'pattern': {'shape': ''}},
              'name': 'reward:collected',
              'offsetgroup': 'reward:collected',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([datetime.datetime(2024, 1, 26, 0, 0),
                          datetime.datetime(2024, 1, 27, 0, 0),
                          datetime.datetime(2024, 1, 28, 0, 0),
                          datetime.datetime(2024, 1, 29, 0, 0),
                          datetime.datetime(2024, 1, 30, 0, 0),
                          datetime.datetime(2024, 1, 31, 0, 0),
                          datetime.datetime(2024, 2, 1, 0, 0),
                          datetime.datetime(2024, 2, 2, 0, 0),
                          datetime.datetime(2024, 2, 3, 0, 0),
                          datetime.datetime(2024, 2, 4, 0, 0),
                          datetime.datetime(2024, 2, 5, 0, 0),
                          datetime.datetime(2024, 2, 6, 0, 0),
                          datetime.datetime(2024, 2, 7, 0, 0),
                          datetime.datetime(2024, 2, 8, 0, 0),
                          datetime.datetime(2024, 2, 9, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([2856775, 2240044, 2207550, 1939910, 1752335, 1501622, 1422716, 1491058,
                          1393885, 1487163, 1232190, 1571227, 2528249,  996157,  891328],
                         dtype=int64),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'event=reward:used<br>date=%{x}<br>coins=%{y}<extra></extra>',
              'legendgroup': 'reward:used',
              'marker': {'color': '#000002', 'pattern': {'shape': ''}},
              'name': 'reward:used',
              'offsetgroup': 'reward:used',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([datetime.datetime(2024, 1, 26, 0, 0),
                          datetime.datetime(2024, 1, 27, 0, 0),
                          datetime.datetime(2024, 1, 28, 0, 0),
                          datetime.datetime(2024, 1, 29, 0, 0),
                          datetime.datetime(2024, 1, 30, 0, 0),
                          datetime.datetime(2024, 1, 31, 0, 0),
                          datetime.datetime(2024, 2, 1, 0, 0),
                          datetime.datetime(2024, 2, 2, 0, 0),
                          datetime.datetime(2024, 2, 3, 0, 0),
                          datetime.datetime(2024, 2, 4, 0, 0),
                          datetime.datetime(2024, 2, 5, 0, 0),
                          datetime.datetime(2024, 2, 6, 0, 0),
                          datetime.datetime(2024, 2, 7, 0, 0),
                          datetime.datetime(2024, 2, 8, 0, 0),
                          datetime.datetime(2024, 2, 9, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([-2040750, -1907950, -1830500, -1559500, -1528800, -1274000, -1171850,
                          -1112850, -1139450, -1350000, -1060400,  -985400,  -820700,  -703700,
                           -667000], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'event'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'coins'}}}
})

In [26]:
ads.isnull().sum().tail(20)

attempt                       62
lvl_name                      97
case_num                    1151
clue_num                    1151
clue_name                   1151
task_num                    1151
attempt_number                 0
device_inventory_name    1506911
ad_type                        0
ad_network                     2
revenue                        2
revenue_precision              2
request_reason                 0
show_reason                   15
ad_placement                  50
split                          0
ab_test_variants               0
retention_day                  0
last_interaction_date          0
days_inactive                  0
dtype: int64

In [65]:
coins = df[(df.event == 'reward:collected')].groupby(['reward_reason'])['coins'].sum().reset_index()
def top_n_pie(df, n, default='others'):
    sorted = df.sort_values(by=['coins'], ascending=False)
    return df.reward_reason.where(df.reward_reason.isin(sorted.reward_reason[:n]), default)


In [71]:
default='others'
n=7
sorted=coins.sort_values(by=['coins'], ascending=False)
coins.reward_reason.where(coins.reward_reason.isin(sorted.reward_reason[:n]), default)

coins.assign(reward_reason= lambda df_:top_n_pie(df_,n=5) )

,reward_reason,coins
0,others,292500
1,others,216000
2,others,60000
3,others,50000
4,coin_100000,3625000
5,others,30000
6,others,180000
7,others,819000
8,level_completed,62934147
9,level_completed_double_reward,3579232


In [76]:
df[df.event=='reward:collected'].reward_reason.value_counts()

def top_n(ser, n, default='others'):
    counts=ser.value_counts()
    return ser.where(ser.isin(counts.index[:n]), default)

reward_reason
level_completed                  1697235
level_completed_tiple_reward       59175
level_completed_double_reward      48323
the_goat                             264
piggy_bank_1_offer                   169
csi_starter_kit                      126
case_1_OTP                            45
special_offer                         45
coin_1000                             30
professional_investigator             29
coin_100000                           29
case_1_special_offer                  24
csi_professional_pack                 15
coin_10000                             4
coin_5000                              4
Name: count, dtype: int64

In [72]:
coins

,reward_reason,coins
0,case_1_OTP,292500
1,case_1_special_offer,216000
2,coin_1000,60000
3,coin_10000,50000
4,coin_100000,3625000
5,coin_5000,30000
6,csi_professional_pack,180000
7,csi_starter_kit,819000
8,level_completed,62934147
9,level_completed_double_reward,3579232


In [3]:
import pandas as pd
from helpers.funnel_helper import *
path = './data/funnel/funnel.csv'
df=pd.read_csv(path)
max_users = df.users.max()
df['Users Percentage %'] = df['users'].apply(lambda x: round(100* (x/max_users) ,2))
df['Step'] = df.apply(lambda x: create_funnel_step(x),axis=1)

In [2]:
import numpy as np
import plotly.express as px

df = df[df.event.isin(['level:started','button:clicked', 'app:init:completed','level:completed']) & df.feature.isin([np.nan, 'tutorial', 'gameplay_tutorial'])].sort_values('users',ascending=False)
fig = px.funnel(df.head(1000), x='Users Percentage %', y='Step')
fig.show()

NameError: name 'df' is not defined

In [18]:
fig


In [10]:
ab_test=pd.read_csv('./data/ab_test/ab_test.csv')
ab_test = (ab_test
           [['progress_id', 'install_date', 'ab_test_variants', 'n_days_active','n_levels', 
             'boosters_used', 'coins_used', 'n_purchases', 'ad_revenue']]
           [ab_test.ab_test_variants
            .isin(['Control', 'Variant A'])
            ].reset_index(drop=True))

control_size = ab_test[ab_test.ab_test_variants=='Control']['progress_id'].nunique()
treatment_size = ab_test[ab_test.ab_test_variants=='Variant A']['progress_id'].nunique()



,progress_id,install_date,ab_test_variants,n_days_active,n_levels,boosters_used,coins_used,n_purchases,ad_revenue
0,0003616b-65c9-4774-b548-d84574a1a759,2024-02-16,Control,2,90,2,2800,0,1.027132
1,0009f46c-ba7a-4ab9-b613-ae8936d032eb,2023-11-21,Control,2,24,0,0,0,0.131149
2,00104466-9e95-4564-81fa-a880414cb983,2024-01-24,Variant A,1,1,0,0,0,0.000000
3,00169026-ac6d-4e6d-ac0a-5a3e9faef388,2024-01-06,Control,1,9,2,0,0,0.192693
4,001b62db-6ab6-4f92-ab3b-c9b22b54000c,2024-01-20,Variant A,1,2,0,0,0,0.014554
...,...,...,...,...,...,...,...,...,...
4261,ff67548b-854f-4bda-a0e3-ed888412bdfd,2024-01-21,Variant A,1,1,0,0,0,0.000000
4262,ff9819f7-b1ae-4c11-b973-00ac41160df1,2024-02-19,Control,2,4,0,0,0,0.000000
4263,ffba2695-8363-4eb7-9f4e-9220f38a3eb0,2024-02-16,Control,1,2,0,0,0,0.027455
4264,ffccc7dd-8f5a-4ab9-9691-99c89822ef27,2024-02-21,Variant A,1,36,2,1800,0,0.122819
